In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn import tree
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score

df = pd.read_csv('./data/train.csv')
test_df  = pd.read_csv('./data/test.csv')
print(df.columns)
print()
print(test_df.columns)

# we need to  encode each categorical column
to_encode = []
for column in df.columns:
    if column == 'LoanID':
        continue
    if df[column].dtype == 'object':
        to_encode.append(column)
    elif df[column].dtype not in ['float64', 'int64', 'float', 'int']:
        to_encode.append(column)
# for column in to_encode:
#     le = LabelEncoder() #for now let's use labelEncoder
#     df[column] = le.fit_transform(df[column])
#     test_df[column] = le.transform(test_df[column])
    # later use le.inverse_transform if needed

#one hot
df = pd.get_dummies(df, columns=to_encode, drop_first=True, dtype=int)

print('done that')

#one  hot
# df = pd.get_dummies(df, columns=to_encode, drop_first=True)

# now all the categorical columns are set
df = df.drop(columns=['LoanID'])

train_df, validation_df = train_test_split(df, test_size=0.2, random_state=17)


x_train_df = train_df.drop(columns=['Default'])
y_train_df = train_df['Default']

x_validation_df = validation_df.drop(columns=['Default'])
y_validation_df = validation_df['Default']


tree_clf = tree.DecisionTreeClassifier()

param_grid = {
    'criterion': ['gini', 'entropy'],  # Split quality measure
    'max_depth': [2, 4, 6, 8, 10, 12, 18, 20, 25, 40],     # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],   # Minimum number of samples required to split a node
    'min_samples_leaf': [1,2]      # Minimum number of samples required to be a leaf node
}

rs = RandomizedSearchCV(estimator=tree_clf, param_distributions=param_grid, random_state=17)
rs.fit(x_train_df, y_train_df)
print("Best Parameters:", rs.best_params_)


# tree_clf.fit(x_train_df, y_train_df)

# y_pred = tree_clf.predict(x_validation_df)
y_pred = rs.predict(x_validation_df)

train_acc = accuracy_score(y_train_df, rs.predict(x_train_df))
valid_acc = accuracy_score(y_validation_df, y_pred)

print(f'Training accuracy {train_acc}')
print(f'Validation accuracy {valid_acc}')




newdf = pd.DataFrame({"LoanID": test_df['LoanID'], "Default": rs.predict(X=test_df.drop(columns=['LoanID']))})
newdf.to_csv('dtree_out.csv', index=False)

Index(['LoanID', 'Age', 'Income', 'LoanAmount', 'CreditScore',
       'MonthsEmployed', 'NumCreditLines', 'InterestRate', 'LoanTerm',
       'DTIRatio', 'Education', 'EmploymentType', 'MaritalStatus',
       'HasMortgage', 'HasDependents', 'LoanPurpose', 'HasCoSigner',
       'Default'],
      dtype='object')

Index(['LoanID', 'Age', 'Income', 'LoanAmount', 'CreditScore',
       'MonthsEmployed', 'NumCreditLines', 'InterestRate', 'LoanTerm',
       'DTIRatio', 'Education', 'EmploymentType', 'MaritalStatus',
       'HasMortgage', 'HasDependents', 'LoanPurpose', 'HasCoSigner'],
      dtype='object')
done that
Best Parameters: {'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': 6, 'criterion': 'gini'}
Training accuracy 0.8854186426469058
Validation accuracy 0.885818484433131


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Education
- EmploymentType
- HasCoSigner
- HasDependents
- HasMortgage
- ...
Feature names seen at fit time, yet now missing:
- Education_High School
- Education_Master's
- Education_PhD
- EmploymentType_Part-time
- EmploymentType_Self-employed
- ...
